In [1]:
import pandas as pd
import json

In [5]:
df_hrsd = pd.read_excel('data/HRSD Resolution Notes Evaluation Sept 6th.xlsx', sheet_name='Sheet1')
df_itsm = pd.read_excel('data/ITSM Resolution Notes Evaluation Sept 7th.xlsx', sheet_name='Sheet1')
df_csm = pd.read_excel('data/CSM Resolution Notes Evaluation Sept 7th.xlsx', sheet_name='Sheet1')
print(df_hrsd.columns)
print(df_itsm.columns)
print(df_csm.columns)

Index(['Index', 'task_id', 'Case', 'NOW LLM Resolution Notes', 'Reviewer',
       'Resolution Notes is Perfect\n(Yes/No)',
       'Missed Important Details\n(1/2/3)', 'Had Made-up Details\n(1/2/3)',
       'Mixed up who said what\n(1/2/3)',
       'Had too much unnecessary info\n(1/2/3)',
       'Has details in wrong sections\n(1/2/3)', 'Golden Resolution Notes',
       'Reviewer's notes', 'Linguist Comments', 'Linguist', 'Unnamed: 15'],
      dtype='object')
Index(['Index', 'Incident Number', 'Incident Data',
       'NOW LLM Generated Resolution Notes', 'Reviewer',
       'Resolution Notes is Perfect\n(Yes/No)',
       'Missed Important Details\n(1/2/3)', 'Had Made-up Details\n(1/2/3)',
       'Mixed up who said what\n(1/2/3)',
       'Had too much unnecessary info\n(1/2/3)', 'Golden Resolution Notes',
       'Reviewer's notes', 'Linguist Comments', 'Linguist'],
      dtype='object')
Index(['Index', 'task_id', 'state', 'Case Data',
       'NOW LLM Generated Resolution Notes', 'Reviewe

In [11]:
output_data = []
id = 0
for index, row in df_csm.iterrows():
    curr_output = {}
    curr_output['inputs_pretokenized'] = row['Case Data']
    if pd.isna(row['Golden Resolution Notes']):
        curr_output['targets_pretokenized'] = ""
    else:
        curr_output['targets_pretokenized'] = row['Golden Resolution Notes']
    curr_output['id'] = str(id)
    curr_output['bu'] = 'csm'
    output_data.append(curr_output)
    id += 1
for index, row in df_itsm.iterrows():
    curr_output = {}
    curr_output['inputs_pretokenized'] = row['Incident Data']
    if pd.isna(row['Golden Resolution Notes']):
        curr_output['targets_pretokenized'] = ""
    else:
        curr_output['targets_pretokenized'] = row['Golden Resolution Notes']
    curr_output['id'] = str(id)
    curr_output['bu'] = 'itsm'
    output_data.append(curr_output)
    id += 1
for index, row in df_hrsd.iterrows():
    curr_output = {}
    curr_output['inputs_pretokenized'] = row['Case']
    if pd.isna(row['Golden Resolution Notes']):
        curr_output['targets_pretokenized'] = ""
    else:
        curr_output['targets_pretokenized'] = row['Golden Resolution Notes']
    curr_output['id'] = str(id)
    curr_output['bu'] = 'hrsd'
    output_data.append(curr_output)
    id += 1
with open('data/bu_resolution_notes_eval.json', 'w') as f:
    json.dump(output_data, f, indent=4)

In [12]:
with open('data/bu_resolution_notes_eval.json', 'r') as f:
    data = json.load(f)
for sample in data:
    sample['inputs_pretokenized'] = sample['inputs_pretokenized'].split('<|end|>\n<|user|>')[0].replace('<|end|>', '')
with open('data/bu_resolution_notes_eval.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
with open('data/bu_resolution_notes_eval.json', 'r') as f:
    data = json.load(f)
cleaned_data = []
for sample in data:
    if sample['targets_pretokenized'] == '':
        continue
    else:
        cleaned_data.append(sample)
with open('data/filtered_bu_resolution_notes_eval.json', 'w') as f:
    json.dump(cleaned_data, f, indent=4)

In [10]:
summac_precision_threshold = 0.5
for prompt_index in ['P1', 'P2', 'P3', 'P4', 'P5']:
    outputs = json.load(open('OUTPUTS/9_5_2023/inputs_{}_outputs.json'.format(prompt_index), 'r'))
    count = 0
    for sample in outputs:
        if sample['summac precision'] >= summac_precision_threshold:
            count += 1
    print('Prompt {} has {} samples with precision >= {}'.format(prompt_index, count, summac_precision_threshold))

Prompt P1 has 53 samples with precision >= 0.5
Prompt P2 has 52 samples with precision >= 0.5
Prompt P3 has 35 samples with precision >= 0.5
Prompt P4 has 47 samples with precision >= 0.5
Prompt P5 has 48 samples with precision >= 0.5
